In [94]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

# Word and Position Embedding implemetation of Transformer

In [95]:
# for word embedding
batch_size = 2
# size of word table 
max_num_src_words = 8
max_num_tgt_words = 8
# model dimension
model_dim = 8
#max length of sequence
#max_src_seq_len = 5
#max_tgt_seq_len = 5
max_position_len = 0


src_len = torch.Tensor([2,4]).to(torch.int32)
tgt_len = torch.Tensor([4,3]).to(torch.int32)

# sentenced constructed from batch of word index, and pad with 0
src_seq = torch.cat([torch.unsqueeze(F.pad(torch.randint(1,max_num_src_words,(L,)),(0,max(src_len)-L)),0) for L in src_len])
tgt_seq = torch.cat([torch.unsqueeze(F.pad(torch.randint(1,max_num_src_words,(L,)),(0,max(tgt_len)-L)),0) for L in tgt_len])
max_position_len = max(src_len)
#test = [torch.unsqueeze(F.pad(torch.randint(1,max_num_src_words,(L,)),(0,max_src_seq_len-L)),0) for L in src_len]
#print(type(test))
#print(src_seq)
#print(tgt_seq)

'''word embedding construction'''
src_embedding_table = nn.Embedding(max_num_src_words+1,model_dim) # +1 for padding 
tgt_embedding_table = nn.Embedding(max_num_tgt_words+1,model_dim)

src_embedding = src_embedding_table(src_seq) # [batch*max_src_len*dim]  2*4*8
tgt_embedding = src_embedding_table(tgt_seq)

#print(src_embedding_table.weight) # 9*8, the first row represent padding
##print(src_embedding)

'''postion embedding construction'''
pos_mat = torch.arange(max(src_len)).reshape(-1,1) # every row is the same, the position of the token in the sequence [4,1]
i_mat = torch.arange(0,model_dim,2).reshape(1,-1)/model_dim  # every column is the same, the dimension  [1,4]
i_mat = torch.pow(10000,i_mat)

pe_embedding_table = torch.zeros(max_position_len,model_dim) #[4,8]
pe_embedding_table[:,0::2] = torch.sin(pos_mat/i_mat) #pos_mat/i_mat = [4,4]
pe_embedding_table[:,1::2] = torch.cos(pos_mat/i_mat) 
# wrap the pe embedding table to embedding layer
pe_embedding = nn.Embedding(max_position_len,model_dim)
pe_embedding.weight = nn.Parameter(pe_embedding_table,requires_grad=False)

src_pos = torch.cat([torch.unsqueeze(torch.arange(max(src_len)),0) for _ in src_len]).to(torch.int32) #[batch,max(scr_len)] 2*4
tgt_pos = torch.cat([torch.unsqueeze(torch.arange(max(tgt_len)),0) for _ in tgt_len]).to(torch.int32)
print(src_pos)
src_pe_embedding = pe_embedding(src_pos)#batch*max(src_len)*dim
tgt_pe_embedding = pe_embedding(tgt_pos)
print(src_pe_embedding)
#print(pos_mat)
#print(i_mat)
print(pe_embedding_table)
#print(pe_embedding.weight)




tensor([[0, 1, 2, 3],
        [0, 1, 2, 3]], dtype=torch.int32)
tensor([[[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
           1.0000e+00,  0.0000e+00,  1.0000e+00],
         [ 8.4147e-01,  5.4030e-01,  9.9833e-02,  9.9500e-01,  9.9998e-03,
           9.9995e-01,  1.0000e-03,  1.0000e+00],
         [ 9.0930e-01, -4.1615e-01,  1.9867e-01,  9.8007e-01,  1.9999e-02,
           9.9980e-01,  2.0000e-03,  1.0000e+00],
         [ 1.4112e-01, -9.8999e-01,  2.9552e-01,  9.5534e-01,  2.9995e-02,
           9.9955e-01,  3.0000e-03,  1.0000e+00]],

        [[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
           1.0000e+00,  0.0000e+00,  1.0000e+00],
         [ 8.4147e-01,  5.4030e-01,  9.9833e-02,  9.9500e-01,  9.9998e-03,
           9.9995e-01,  1.0000e-03,  1.0000e+00],
         [ 9.0930e-01, -4.1615e-01,  1.9867e-01,  9.8007e-01,  1.9999e-02,
           9.9980e-01,  2.0000e-03,  1.0000e+00],
         [ 1.4112e-01, -9.8999e-01,  2.9552e-01,  9.5534e-0

In [96]:
# reason why we need scaled before softmax
alpha1 = 0,1
alpha2 = 10
score = torch.randn(5)
prob = F.softmax(score,-1)
def softmax_func(score):
    return F.softmax(score)
jaco_mat = torch.autograd.functional.jacobian(softmax_func,score)
print(jaco_mat)
print(score)
print(prob)

tensor([[ 0.0395, -0.0062, -0.0055, -0.0025, -0.0253],
        [-0.0062,  0.1283, -0.0200, -0.0091, -0.0929],
        [-0.0055, -0.0200,  0.1149, -0.0080, -0.0815],
        [-0.0025, -0.0091, -0.0080,  0.0566, -0.0370],
        [-0.0253, -0.0929, -0.0815, -0.0370,  0.2368]])
tensor([-1.3812, -0.0819, -0.2138, -1.0017,  1.3214])
tensor([0.0412, 0.1511, 0.1325, 0.0602, 0.6149])


/var/folders/_s/qf2hpr911z5167gwmr6bb1940000gn/T/ipykernel_29846/2366350606.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(score)


# encoder self-attention mask construction

In [100]:
'''
shape of mask: [batch_size, max_src_len,max_src_len],
value is 1 or -inf
'''
valid_encoder_pos = torch.cat([torch.unsqueeze(F.pad(torch.ones(L),(0,max(src_len)-L)),0) for L in src_len])
print(valid_encoder_pos)
valid_encoder_pos = torch.unsqueeze(valid_encoder_pos,2) #[batch_size,max(src_len),1]
#print(valid_encoder_pos.shape)
valid_encoder_pos_matrix = torch.bmm(valid_encoder_pos,valid_encoder_pos.transpose(1,2))
#print(valid_encoder_pos_matrix)
invalid_encoder_pos_matrix = 1-valid_encoder_pos_matrix
#print(invalid_encoder_pos_matrix)
mask_encoder_self_attention = invalid_encoder_pos_matrix.to(torch.bool)
#print(mask_encoder_self_attention.shape)

score = torch.randn(batch_size,max(src_len),max(src_len))
#print(score)
masked_score = score.masked_fill(mask_encoder_self_attention,-1e9)
#print(masked_score.shape)
prob = F.softmax(masked_score,-1)
#print(prob)



tensor([[1., 1., 0., 0.],
        [1., 1., 1., 1.]])
torch.Size([2, 4, 4])
torch.Size([2, 4, 4])
tensor([[[0.2545, 0.7455, 0.0000, 0.0000],
         [0.7336, 0.2664, 0.0000, 0.0000],
         [0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500]],

        [[0.1371, 0.2022, 0.0548, 0.6058],
         [0.2571, 0.2074, 0.3833, 0.1522],
         [0.1210, 0.0433, 0.2453, 0.5905],
         [0.5462, 0.0928, 0.1048, 0.2563]]])


# Decoder Mask

In [98]:
'''
cross-attention mask A@K^T shape: [batch_size, tgt_seq_len,src_seq_len]
'''
valid_encoder_pos = torch.cat([torch.unsqueeze(F.pad(torch.ones(L),(0,max(src_len)-L)),0) for L in src_len])
#print(valid_encoder_pos)
valid_encoder_pos = torch.unsqueeze(valid_encoder_pos,2) #[batch_size,max(src_len),1]

valid_decoder_pos = torch.cat([torch.unsqueeze(F.pad(torch.ones(L),(0,max(tgt_len)-L)),0) for L in tgt_len])

#print(valid_encoder_pos)
valid_decoder_pos = torch.unsqueeze(valid_decoder_pos,2) #[batch_size,max(src_len),1]

valid_cross_pos_matrix = torch.bmm(valid_decoder_pos,valid_encoder_pos.transpose(1,2))
#print(valid_decoder_pos.shape)
invalid_cross_pos_matrix = 1-valid_cross_pos_matrix
mask_cross_attention = invalid_encoder_pos_matrix.to(torch.bool)
#print(valid_cross_pos_matrix)
#print(invalid_cross_pos_matrix)
#print(mask_cross_attention)

'''
Decoder self-attention mask
'''
valid_decoder_tri_matrix = torch.cat([torch.unsqueeze(F.pad(torch.tril(torch.ones(L,L)),(0,max(tgt_len)-L,0,max(tgt_len)-L)),0) for L in tgt_len])
#print(valid_decoder_tri_matrix)
invalid_decoder_tri_matrix = 1-valid_decoder_tri_matrix
invalid_decoder_tri_matrix = invalid_decoder_tri_matrix.to(torch.bool)
print(invalid_decoder_tri_matrix)

score = torch.randn(batch_size,max(tgt_len),max(tgt_len))
masked_score = score.masked_fill(invalid_decoder_tri_matrix,-1e9)
prob = F.softmax(masked_score,-1)
print(prob)

tensor([[[False,  True,  True,  True],
         [False, False,  True,  True],
         [False, False, False,  True],
         [False, False, False, False]],

        [[False,  True,  True,  True],
         [False, False,  True,  True],
         [False, False, False,  True],
         [ True,  True,  True,  True]]])
tensor([[[1.0000, 0.0000, 0.0000, 0.0000],
         [0.6494, 0.3506, 0.0000, 0.0000],
         [0.5234, 0.1916, 0.2849, 0.0000],
         [0.1583, 0.3316, 0.3902, 0.1199]],

        [[1.0000, 0.0000, 0.0000, 0.0000],
         [0.7329, 0.2671, 0.0000, 0.0000],
         [0.0760, 0.1678, 0.7562, 0.0000],
         [0.2500, 0.2500, 0.2500, 0.2500]]])


# Self-Attention

In [99]:
def scaled_dot_product_attention(Q,K,V,atten_mask):
    # shape of Q,K,V: (batch_size*num_head,seq_len,model_dim/num_head)
    score = torch.bmm(Q,K.transpose(-2,-1))/torch.sqrt(model_dim)
    masked_score = score.masked_fill( atten_mask,-1e9)
    prob = F.softmax(masked_score,-1)
    context = torch.bmm(prob,V)
    return context